In [1]:
for t1 in range(49):

    import numpy as np
    import math
    import scipy
    from scipy import stats
    import random
    from tqdm import tqdm
    import multiprocessing as mp
    import datetime
    import matplotlib
    import matplotlib.pyplot as plt
    from scipy.optimize import fsolve
    import warnings
    warnings.filterwarnings("ignore")

    cores = 64
    rm = 20
    dt = 4
    dm = 5
    V = 1000 # 单位：L
    N = 300*1000*10 # 每立方厘米，故乘1000
    mu = 4.45
    sigma = 1.3
    eps = 10**-2
    fHz = 50
    Q0 = 110
    sound_open = 0

    mm = rm**3
    m = np.array((range(dm, mm+dm, dm)))
    n = len(m)
    r = m**(1/3)


    def count_y(Q,f,r0):
        r = r0/10**6
        q = 10**((Q-120)/10)
        ita = 1.79*10**-5
        omiga = 2*np.pi*f
        Va = np.sqrt(2*q/1.29/330)
        tau = 2*1000*r**2/9/ita
        Y = 2*Va*np.cos(np.arctan(omiga*tau))/omiga/np.sqrt(1+omiga**2*tau**2)
        return Y

    def E(r1,r2):
        E = (-2.93811594e-04*(r1**3+r2**3)+3.20089591e-02*(r1**2+r2**2)+ 3.19191002e-01*(r1+r2)-6.83720963e-05*(r1**2*r2+r2**2*r1)
        +1.05130783e-02*r1*r2-6.47710095)/100
        if E<0.001:
            E=0.001
        if E>1:
            E=1
        return E

    def Sound(Q,f,r1,r2):
        return 4*np.pi*f*(r1/10**6+r2/10**6)**2*np.abs(count_y(Q,f,r1)-count_y(Q,f,r2))*E(r1,r2)*1000/V   # 原来单位是m3/s，故乘以1000

    def KAW(Q0,f,r10,r20):
        r1 = r10/10**6
        r2 = r20/10**6
        L = 0.001
        miu = 1.818*10**-5
        Q = 10**((Q0-120)/10)
        omiga = 2*np.pi*f
        tau1 = 2*1000*r1**2/9/miu
        tau2 = 2*1000*r2**2/9/miu
        ua = np.sqrt(2*Q/1.29/330)
        h1 = 2*ua/9/np.pi/omiga/r1*1.29/1000
        h2 = 2*ua/9/np.pi/omiga/r2*1.29/1000
        n1 = omiga*tau1/np.sqrt(1+omiga**2*tau1**2)
        n2 = omiga*tau2/np.sqrt(1+omiga**2*tau2**2)
        m1 = n1/(1+2*h1*n1**2)
        m2 = n2/(1+2*h2*n2**2)
        K = 3/2/L*ua*(r1+r2)**2*(r1*m1+r2*m2)
        return K*10**3/V*E(r1,r2)


    vp0 = np.zeros(len(r))
    for i in range(len(r)):
        miu = 1.818*10**-4
        g = 980
        def func(x):
            return 4/3*np.pi*(r[i]/10**4)**3*g-6*np.pi*miu*(r[i]/10**4)*x*(1+0.15*(2*(r[i]/10**4)*1.29*10**-3*x/miu)**0.687) 
        vp0[i] = fsolve(func,10)

    def vp(r):
        if r>20*10**-4:
            return 2*1000*(r/10**4)**2/9/1.29/(1.818*10**-5)*9.81*100
        else:
            return vp0[round((r*10**4)**3/dm)-1]

    def count_g(r,eps,u):
        niu = 0.14
        miu = 1.818*10**-4
        g = 980
        taus = vp(r)/g
        tauk = np.sqrt(niu/eps)
        Rg = u**2*np.sqrt(15/niu/eps)
        St = taus/tauk
        y0 = 18*St**2
        y1 = 0.36*St**2.5*np.exp(-St**2.5)
        y2 = 0.24*np.exp(-0.5*St)
        y3 = 0.013*np.exp(-0.07*St)
        z0 = 1/2*(1+np.tanh((St-0.5)/0.25))
        z1 = 1/2*(1+np.tanh((St-1.25)/0.1))
        z2 = 1/2*(1+np.tanh((St-6.5)/2.5))
        g = 1+y0*(1-z0**2)+z0**2*Rg*(y1*(1-z1)+y2*z1+y3*z2)
        return g

    def Turb(r10,r20,eps0):
        eps = eps0*10**4
        u = 202
        g = 980
        niu = 0.14
        miu = 1.818*10**-4
        r1 = r10/10**4
        r2 = r20/10**4
        R = r1+r2
        vk = (niu*eps)**(1/4)
        ita = (niu**3/eps)**(1/4)
        shear = 1/15*vk**2*(R/ita)**2

        TL = u**2/eps
        tausi = vp(r1)/g
        tausj = vp(r2)/g
        thi = 2.5*tausi/TL
        thj = 2.5*tausj/TL
        z = max(thi/thj,thj/thi)
        Cw = 1+0.6*np.exp(-(z-1)**1.5)
        gam = z*0.183*u**2/np.sqrt(eps*niu)
        accel0 = u**2*gam/(gam-1)*(thi+thj-4*thi*thj/(thi+thj)*np.sqrt((1+thi+thj)/(1+thi)/(1+thj)))*(1/(1+thi)/(1+thj)-1/(1+gam*thi)/(1+gam*thj))
        accel = accel0*Cw

        phi = max(tausj,tausi)/min(tausj,tausi)
        rou = 2.6*np.exp(-phi)+0.205*np.exp(-0.0206*phi)/2*(1+np.tanh(phi-3))
        gij = 1+rou*np.sqrt((count_g(r1,eps,u)-1)*(count_g(r2,eps,u)-1))
        w = np.sqrt(2/np.pi*(shear+accel))
        K = ((accel+shear)*gij**2*2/np.pi)**(1/2)*2*np.pi*R**2
        return K/10**3/V   #原文单位全是cm

    def Kernel(r1,r2,Q,f,eps):
        if sound_open == 0:
            k = Turb(r1, r2,eps)
        else:
            k = (Turb(r1, r2,eps)**2 + Sound(Q0,fHz,r1,r2)**2 + KAW(Q0,fHz,r1,r2)**2)**(1/2)
        return k


    def kernel_mat(r1):
        k = [0]*n
        for i in range(round(r1**3/dm)-1):
            k[i] = Kernel(r1, r[i],Q0,fHz,eps)
        return k


    def lognormal():
        f = stats.lognorm.pdf(x=r,s=np.log(sigma),scale=mu)*V*N/3*r**-2
        return f



    f = lognormal() * dm # Golovin分布中的峰值
    f_alltime = np.zeros((13,n))
    drop = np.zeros(62)
    f_alltime[0] = f/dm
    R = np.array([0]*10000)
    F = np.array([0]*10000)
    M = sum(m*f)+sum(R**3*F)

    start = [int(i*n/cores) for i in range(cores)]
    end = [int(i*n/cores) for i in range(1,cores)]+[n]
    mid = (np.array(start)+np.array(end))/2
    start_new = [int(n/rm*i) for i in range(rm)]
    end_new = [int(n/rm*i) for i in range(1,rm+1)]
    start_inter = [0]+[n/rm/2*i+n/rm/4 for i in range(1,2*rm-1)]
    end_inter = [n/rm/2*i+n/rm/4 for i in range(1,2*rm-1)]+[n]
    mid_inter = [n/rm/2*i for i in range(1,2*rm)]

    if __name__ == '__main__':
        P = mp.Pool(processes = cores)
        k = [P.apply_async(kernel_mat, args=(i,)) for i in r]
        k = np.array([p.get() for p in tqdm(k)])
    k = k + k.T
    for i in range(n):
        k[i,i] = Kernel(r[i], r[i],Q0,fHz,eps)

    k_add = [0]*n
    k_add[0] = np.array([0])
    for i in range(n):
        k_add[i]=np.diag(np.fliplr(k), n-i)
    k_new = [0]*n
    for i in range(n):
        k_new[i] = np.diag(np.fliplr(k), -i)
    k = [k[start[i]:end[i]] for i in range(cores)]
    k_add = [k_add[start[i]:end[i]] for i in range(cores)]
    k_new = [k_new[start_new[i]:end_new[i]] for i in range(len(start_new))]

    def count_small(index):
        main_f = np.frombuffer(f,dtype='f')
        main_R = np.frombuffer(R,dtype='f')
        main_F = np.frombuffer(F,dtype='i')
        main_not01 = np.frombuffer(not0,dtype='i')
        main_not0 = np.where(main_not01!=0)[0]
        addend = [0]*n
        addend[0] = np.array([0])
        for i in range(1,n):
            addend[i] = main_f[0:i]
        l = end[index] - start[index]
        increase_s = np.zeros(l)
        increase_b = np.zeros(l)
        for i in range(l):
            increase_s[i] = sum(addend[start[index]+i][0:int((start[index]+i)/2)]*np.flipud(addend[start[index]+i][int((start[index]+i+1)/2):])*k_add[index][i][0:int((start[index]+i)/2)])-sum(k[index][i]*main_f[start[index]+i]*main_f)
            if (start[index]+i)%2 != 0:
                increase_s[i] += k_add[index][i][int((start[index]+i)/2)]*addend[start[index]+i][int((start[index]+i)/2)]**2/2
            increase_b[i] = sum([main_f[start[index]+i]*main_F[main_not0[j]]*Kernel(main_R[main_not0[j]],r[start[index]+i],Q0,fHz,eps) for j in range(len(main_R[main_not0]))])
        main_f[start[index]:end[index]] += (increase_s-increase_b)*dt


    def count_inter(index):
        main_f = np.frombuffer(f,dtype='f')
        main_R = np.frombuffer(R,dtype='f')
        main_F = np.frombuffer(F,dtype='i')
        main_not01 = np.frombuffer(not0,dtype='i')
        main_not0 = np.where(main_not01!=0)[0]  
        main_p_M = np.frombuffer(p_M,dtype='f')
        for i in main_not0:
            random.seed(datetime.datetime.now().timestamp())
            phi = random.random()
            p = Kernel(main_R[main_not0[i]],(mid_inter[index]*dm)**(1/3),Q0,fHz,eps)*main_f[round(mid_inter[index])]*(end_inter[index]-start_inter[index])*dt/main_p_M
            if phi < p:
                main_R[i] = (main_R[i]**3+mid_inter[index]*dm)**(1/3)
        return


    def count_big(index):
        main_R = np.frombuffer(R,dtype='f')
        main_F = np.frombuffer(F,dtype='i')
        main_not01 = np.frombuffer(not0,dtype='i')
        main_not0 = np.where(main_not01!=0)[0]
        index_0 = np.where(main_not01==0)[0]
        START = [int(i*len(main_not0)/cores) for i in range(cores)]
        END = [int(i*len(main_not0)/cores) for i in range(1,cores)]+[len(main_not0)]

        for i in range(START[index],END[index]):
            for j in range(i):
                if main_F[main_not0[i]]>0 and main_F[main_not0[j]]>0:
                    p = max(main_F[main_not0[i]], main_F[main_not0[j]]) * Kernel(main_R[main_not0[i]],main_R[main_not0[j]],Q0,fHz,eps)*dt
                    random.seed(datetime.datetime.now().timestamp())
                    phi = random.random()
                    if phi < p:
                        if main_F[main_not0[i]] < main_F[main_not0[j]]:
                            main_R[main_not0[i]] = (main_R[main_not0[i]]**3+main_R[main_not0[j]]**3)**(1/3)
                            main_F[main_not0[j]] -= main_F[main_not0[i]]
                        else:
                            main_R[main_not0[j]] = (main_R[main_not0[j]]**3+main_R[main_not0[i]]**3)**(1/3)
                            main_F[main_not0[i]] -= main_F[main_not0[j]]
            if main_F[main_not0[i]]>1:
                p = main_F[main_not0[i]]*(main_F[main_not0[i]]-1)/2 * Kernel(main_R[main_not0[i]],main_R[main_not0[i]],Q0,fHz,eps)*dt
                random.seed(datetime.datetime.now().timestamp())
                phi = random.random()
                if p > 1:
                    main_F[index_0[i]] = int(p)
                    main_R[index_0[i]] = (main_R[main_not0[i]]**3*2)**(1/3)
                    main_F[main_not0[i]] -= int(p)*2
                if phi < p-int(p):
                    main_F[index_0[i]] += 1
                    main_R[index_0[i]] = (main_R[main_not0[i]]**3*2)**(1/3)
                    main_F[main_not0[i]] -= 2
        return


    def add_new(index):
        main_f = np.frombuffer(f,dtype='f')
        main_F = np.frombuffer(F,dtype='i')
        main_R = np.frombuffer(R,dtype='f')
        main_not01 = np.frombuffer(not0,dtype='i')
        index_0 = np.where(main_not01 == 0)[0]
        main_p_M = np.frombuffer(p_M,dtype='f')
        GOAL = np.arange(mm+mm/rm/2, 2*mm, mm/rm)
        addend = [0]*n
        for i in range(n):
            addend[i] = main_f[i:n]
        l = end_new[index] - start_new[index]
        p_comp = np.zeros(l)
        for i in range(l):
            p_comp[i] = sum(addend[start_new[index]+i][0:int((n+1-start_new[index]-i)/2)]*np.flipud(addend[start_new[index]+i][int((n-start_new[index]-i)/2):])*k_new[index][i][0:int((n+1-start_new[index]-i)/2)])*(mm+1+start_new[index]*dm+i*dm)
            if (start_new[index]+i) %2 != 0:
                p_comp[i] -= k_new[index][i][int((n+1-start_new[index]-i)/2)-1]*addend[start_new[index]+i][int((n+1-start_new[index]-i)/2-1)]**2*(mm+1+start_new[index]*dm+i*dm)/2
        p=sum(p_comp)/GOAL[index]*dt/main_p_M
        random.seed(datetime.datetime.now().timestamp())     
        phi = random.random()
        if p > 1:
            main_R[index_0[index]] = GOAL[index]**(1/3)
            main_F[index_0[index]] = int(p)
        if phi < p-int(p):
            main_R[index_0[index]] = GOAL[index]**(1/3)
            main_F[index_0[index]] += 1
        return


    def arrange_big():
        main_F = np.frombuffer(F,dtype='i')
        main_R = np.frombuffer(R,dtype='f')
        main_not0 = np.where(main_F != 0)[0]
        main_dropm = np.frombuffer(dropm,dtype='f')
        F1 = main_F[main_not0]
        R1 = np.round(main_R[main_not0]**3) 
        uR = np.unique(R1)
        uR1 = uR[uR >=43.725**3]   # 31.705 43.725   49.55
        uF1 = np.array([sum(F1[R1 == i]) for i in uR1])
        main_dropm += sum(uR1*uF1)
        uR = uR[uR < 43.725**3]
        uF = np.array([sum(F1[R1 == i]) for i in uR])
        main_F[:len(uF)] = uF
        main_F[len(uF):] = 0
        main_R[:len(uF)] = uR**(1/3)
        main_R[len(uF):] = 0
        return


    def set0():
        main_F = np.frombuffer(F,dtype='i')
        main_not0 = np.frombuffer(not0,dtype='i')
        main_not0[np.where(main_F != 0)[0]] = 1
        main_not0[np.where(main_F == 0)[0]] = 0
        return

    def set_pM():
        main_f = np.frombuffer(f,dtype='f')
        main_F = np.frombuffer(F,dtype='i')
        main_R = np.frombuffer(R,dtype='f')
        main_p_M = np.frombuffer(p_M,dtype='f')
        main_dropm = np.frombuffer(dropm,dtype='f')
        main_not01 = np.frombuffer(not0,dtype='i')
        main_not0 = np.where(main_not01!=0)[0]
        main_p_M[0] = (sum(main_f*m)+sum(main_R[main_not0]**3*main_F[main_not0])+main_dropm)/M
        return





    if __name__ == '__main__':
        R_alltime = [[]]
        F_alltime = [[]]
        f = mp.Array('f', f, lock=False)
        R = mp.Array('f', R, lock=False)
        F = mp.Array('i', F, lock=False)
        dropm = mp.Value('f', 0.0, lock=False)
        not0 = mp.Array('i', [0]*18000, lock=False)
        p_M = mp.Value('f', 1.0, lock=False)
        pool = mp.Pool(cores)
        for t in tqdm(np.arange(dt, 3600+dt,dt)):
            pool.map(count_small, range(cores))
            pool.map(count_inter, range(len(mid_inter)))
            pool.map(count_big, range(cores))
            set0()
            pool.map(add_new,range(len(start_new)))
            arrange_big()
            set0()
            if t%300==0:
                f_alltime[int(t/300)] = np.frombuffer(f,dtype='f')/dm
                R = np.frombuffer(R,dtype='f')
                F = np.frombuffer(F,dtype='i')
                R_alltime += [R[np.where(R!=0)]]
                F_alltime += [F[np.where(F!=0)]]
            if t%60 ==0:
                drop[int(t/60)] = np.frombuffer(dropm,dtype='f')
            if np.frombuffer(dropm,dtype='f') == 0 :
                drop_start = t+dt
            drop[-1] = drop_start
        print(drop_start,(sum(np.frombuffer(f,dtype='f')*m)+sum(R**3*F)+np.frombuffer(dropm,dtype='f'))/M)
        
        
        f_multi = np.loadtxt(r'cont/10t20.txt')
        R_multi = np.loadtxt(r'cont/10t20rb.txt')
        F_multi = np.loadtxt(r'cont/10t20fb.txt')
        drop_multi = np.loadtxt(r'cont/10t20drop.txt')
        f_multi1 = np.concatenate((f_multi, f_alltime),axis=0)
        drop_multi1 = np.concatenate((drop_multi, drop),axis=0)
        R_multi1 = [R_multi[i][np.where(R_multi[i]!=0)] for i in range(len(R_multi))] + R_alltime
        F_multi1 = [F_multi[i][np.where(F_multi[i]!=0)] for i in range(len(R_multi))] + F_alltime
        R_write = np.zeros([len(R_multi1),len(max(R_multi1,key = lambda x: len(x)))])
        for i,j in enumerate(R_multi1):
            R_write[i][0:len(j)] = j
        F_write = np.zeros([len(F_multi1),len(max(F_multi1,key = lambda x: len(x)))])
        for i,j in enumerate(F_multi1):
            F_write[i][0:len(j)] = j
        np.savetxt(r'cont/10t20.txt',f_multi1,fmt='%.3e')
        np.savetxt(r'cont/10t20rb.txt',R_write,fmt='%.3e')
        np.savetxt(r'cont/10t20fb.txt',F_write,fmt='%.1f')
        np.savetxt(r'cont/10t20drop.txt',drop_multi1,fmt='%.3e')
        %reset -f
        

100%|█████████████████████████████████████████████████████████████████████████████████| 900/900 [03:04<00:00,  4.88it/s]


3604 [0.9919822]


100%|█████████████████████████████████████████████████████████████████████████████████| 900/900 [03:12<00:00,  4.68it/s]


3604 [0.99919]


 32%|█████████████████████████▉                                                       | 288/900 [00:34<01:35,  6.40it/s]

In [4]:
i

1599